In [1]:
# ===== CONFIGURATION BLOCK =====
# Input and output folders
INPUT_FOLDER = "People_Input"
OUTPUT_FOLDER = "People_Output"

# Load environment variables
from dotenv import load_dotenv, find_dotenv
import os


# Load .env and override any existing environment variables so changes are picked up without restarting the kernel
load_dotenv(find_dotenv(), override=True)

REGION = os.getenv('REGION')
API_KEY_FACE = os.getenv('API_KEY_FACE')
URL_FACE = os.getenv('URL_FACE')

print('Using face endpoint:', URL_FACE)
print('Face API key present:', bool(API_KEY_FACE))

Using face endpoint: https://facefrance.cognitiveservices.azure.com/
Face API key present: True


In [2]:
# ===== INSTALLATION AND IMPORTS =====
# Run this cell first if packages are not installed
# !pip install azure-cognitiveservices-vision-face python-dotenv pillow

import os
import shutil
import json
from pathlib import Path
from azure.cognitiveservices.vision.face import FaceClient
from azure.cognitiveservices.vision.face.models import TrainingStatusType
from msrest.authentication import CognitiveServicesCredentials
from PIL import Image
import time
import uuid

# Initialize Face client
face_client = FaceClient(URL_FACE, CognitiveServicesCredentials(API_KEY_FACE))

print(f"✓ Connected to Azure Face API in {REGION}")


✓ Connected to Azure Face API in francecentral


In [7]:
# ===== PERSON GROUP SETUP =====
# Create and train a PersonGroup with known faces

PERSON_GROUP_ID = str(uuid.uuid4())  # Unique ID for this person group

def create_person_group(group_id, group_name, known_faces_folder="Known_Faces"):
    """
    Create a PersonGroup and train it with known faces.
    
    Folder structure expected:
    Known_Faces/
        ├── PersonName1/
        │   ├── photo1.jpg
        │   ├── photo2.jpg
        ├── PersonName2/
        │   ├── photo1.jpg
        │   └── photo2.jpg
    """
    try:
        # Delete existing group if it exists
        try:
            face_client.person_group.delete(person_group_id=group_id)
            print(f"Deleted existing PersonGroup: {group_id}")
        except:
            pass
        
        # Create new PersonGroup
        face_client.person_group.create(
            person_group_id=group_id,
            name=group_name,
            recognition_model='recognition_04'
        )
        print(f"✓ Created PersonGroup: {group_name}")
        
        person_map = {}  # Map person_id to name
        
        # Check if Known_Faces folder exists
        if not os.path.exists(known_faces_folder):
            print(f"⚠ Warning: {known_faces_folder} folder not found. Skipping custom face training.")
            print(f"   Create a '{known_faces_folder}' folder with subfolders named after people.")
            return person_map
        
        # Add persons and their faces
        person_folders = [f for f in os.listdir(known_faces_folder) 
                         if os.path.isdir(os.path.join(known_faces_folder, f))]
        
        if not person_folders:
            print(f"⚠ No person folders found in {known_faces_folder}")
            return person_map
        
        for person_name in person_folders:
            person_folder = os.path.join(known_faces_folder, person_name)
            
            # Create person in the group
            person = face_client.person_group_person.create(
                person_group_id=group_id,
                name=person_name
            )
            person_map[person.person_id] = person_name
            print(f"  Added person: {person_name}")
            
            # Add face images for this person
            image_files = [f for f in os.listdir(person_folder) 
                          if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
            
            for image_file in image_files:
                image_path = os.path.join(person_folder, image_file)
                with open(image_path, 'rb') as image_stream:
                    try:
                        face_client.person_group_person.add_face_from_stream(
                            person_group_id=group_id,
                            person_id=person.person_id,
                            image=image_stream,
                            detection_model='detection_03'
                        )
                        print(f"    ✓ Added face: {image_file}")
                    except Exception as e:
                        print(f"    ✗ Error adding {image_file}: {str(e)}")
        
        # Train the PersonGroup
        print(f"\n🔄 Training PersonGroup...")
        face_client.person_group.train(person_group_id=group_id)
        
        # Wait for training to complete
        while True:
            training_status = face_client.person_group.get_training_status(group_id)
            print(f"   Training status: {training_status.status}")
            
            if training_status.status == TrainingStatusType.succeeded:
                print("✓ Training completed successfully!")
                break
            elif training_status.status == TrainingStatusType.failed:
                print(f"✗ Training failed: {training_status.message}")
                break
            
            time.sleep(1)
        
        return person_map
    
    except Exception as e:
        print(f"Error creating PersonGroup: {str(e)}")
        return {}

# Create and train the PersonGroup
person_map = create_person_group(PERSON_GROUP_ID, "Student Face Recognition Group")


Error creating PersonGroup: (InvalidRequest) Invalid request has been sent.


In [4]:
# ===== MAIN PROCESSING FUNCTION =====

def process_photos_with_face_detection(input_folder, output_folder):
    """
    Process photos and organize them by detected faces.
    """
    
    # Create output folder
    os.makedirs(output_folder, exist_ok=True)
    
    # Statistics
    stats = {
        'photos_processed': 0,
        'unique_faces_detected': set(),
        'total_faces_detected': 0,
        'face_names': []
    }
    
    # Metadata storage
    metadata = {}
    
    # Get all image files
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp')
    image_files = [f for f in os.listdir(input_folder) 
                   if f.lower().endswith(image_extensions)]
    
    print(f"\n📁 Processing {len(image_files)} images from {input_folder}...\n")
    
    for idx, image_file in enumerate(image_files, 1):
        image_path = os.path.join(input_folder, image_file)
        print(f"[{idx}/{len(image_files)}] Processing: {image_file}")
        
        try:
            # Detect faces in the image
            with open(image_path, 'rb') as image_stream:
                detected_faces = face_client.face.detect_with_stream(
                    image=image_stream,
                    detection_model='detection_03',
                    recognition_model='recognition_04',
                    return_face_id=True,
                    return_face_attributes=['age', 'gender', 'emotion', 'smile', 'glasses']
                )
            
            stats['photos_processed'] += 1
            face_count = len(detected_faces)
            
            if face_count == 0:
                # No faces detected
                print(f"  ℹ No faces detected")
                no_face_folder = os.path.join(output_folder, "No_Faces_Detected")
                os.makedirs(no_face_folder, exist_ok=True)
                shutil.copy2(image_path, os.path.join(no_face_folder, image_file))
                
                metadata[image_file] = {
                    'faces_detected': 0,
                    'faces': []
                }
                continue
            
            print(f"  ✓ Detected {face_count} face(s)")
            stats['total_faces_detected'] += face_count
            
            # Identify faces if PersonGroup was trained
            identified_faces = []
            face_ids = [face.face_id for face in detected_faces]
            
            if person_map and face_ids:
                try:
                    identify_results = face_client.face.identify(
                        face_ids=face_ids,
                        person_group_id=PERSON_GROUP_ID,
                        max_num_of_candidates_returned=1,
                        confidence_threshold=0.5
                    )
                    
                    for face, identify_result in zip(detected_faces, identify_results):
                        if identify_result.candidates:
                            # Known person identified
                            person_id = identify_result.candidates[0].person_id
                            confidence = identify_result.candidates[0].confidence
                            person_name = person_map[person_id]
                            
                            identified_faces.append({
                                'face_id': str(face.face_id),
                                'person_name': person_name,
                                'confidence': confidence,
                                'type': 'known_person',
                                'attributes': {
                                    'age': face.face_attributes.age if face.face_attributes else None,
                                    'gender': face.face_attributes.gender if face.face_attributes else None,
                                    'emotion': face.face_attributes.emotion.as_dict() if face.face_attributes and face.face_attributes.emotion else None,
                                    'smile': face.face_attributes.smile if face.face_attributes else None,
                                    'glasses': face.face_attributes.glasses if face.face_attributes else None
                                }
                            })
                            
                            print(f"    → Identified: {person_name} (confidence: {confidence:.2f})")
                            stats['unique_faces_detected'].add(person_name)
                            stats['face_names'].append(person_name)
                            
                            # Copy to person folder
                            person_folder = os.path.join(output_folder, person_name)
                            os.makedirs(person_folder, exist_ok=True)
                            shutil.copy2(image_path, os.path.join(person_folder, image_file))
                        else:
                            # Unknown face
                            face_id_short = str(face.face_id)[:8]
                            identified_faces.append({
                                'face_id': str(face.face_id),
                                'person_name': f"Unknown_{face_id_short}",
                                'confidence': 0.0,
                                'type': 'unknown',
                                'attributes': {
                                    'age': face.face_attributes.age if face.face_attributes else None,
                                    'gender': face.face_attributes.gender if face.face_attributes else None,
                                    'emotion': face.face_attributes.emotion.as_dict() if face.face_attributes and face.face_attributes.emotion else None,
                                    'smile': face.face_attributes.smile if face.face_attributes else None,
                                    'glasses': face.face_attributes.glasses if face.face_attributes else None
                                }
                            })
                            
                            print(f"    → Unknown face: {face_id_short}")
                            stats['unique_faces_detected'].add(f"Unknown_{face_id_short}")
                            
                            # Copy to unknown face folder
                            unknown_folder = os.path.join(output_folder, f"Unknown_{face_id_short}")
                            os.makedirs(unknown_folder, exist_ok=True)
                            shutil.copy2(image_path, os.path.join(unknown_folder, image_file))
                
                except Exception as e:
                    print(f"  ⚠ Error identifying faces: {str(e)}")
                    # Treat all as unknown
                    for face in detected_faces:
                        face_id_short = str(face.face_id)[:8]
                        unknown_folder = os.path.join(output_folder, f"Unknown_{face_id_short}")
                        os.makedirs(unknown_folder, exist_ok=True)
                        shutil.copy2(image_path, os.path.join(unknown_folder, image_file))
            else:
                # No PersonGroup trained, use face IDs
                for face in detected_faces:
                    face_id_short = str(face.face_id)[:8]
                    identified_faces.append({
                        'face_id': str(face.face_id),
                        'person_name': f"Face_{face_id_short}",
                        'type': 'unidentified',
                        'attributes': {
                            'age': face.face_attributes.age if face.face_attributes else None,
                            'gender': face.face_attributes.gender if face.face_attributes else None
                        }
                    })
                    
                    stats['unique_faces_detected'].add(f"Face_{face_id_short}")
                    
                    # Copy to face ID folder
                    face_folder = os.path.join(output_folder, f"Face_{face_id_short}")
                    os.makedirs(face_folder, exist_ok=True)
                    shutil.copy2(image_path, os.path.join(face_folder, image_file))
            
            # Store metadata
            metadata[image_file] = {
                'faces_detected': face_count,
                'faces': identified_faces
            }
            
        except Exception as e:
            print(f"  ✗ Error processing {image_file}: {str(e)}")
            stats['photos_processed'] += 1
    
    # Save metadata to JSON
    metadata_path = os.path.join(output_folder, 'face_metadata.json')
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, indent=2, ensure_ascii=False)
    
    print(f"\n✓ Metadata saved to: {metadata_path}")
    
    # Print summary
    print("\n" + "="*60)
    print("📊 PROCESSING SUMMARY")
    print("="*60)
    print(f"Number of photos processed: {stats['photos_processed']}")
    print(f"Number of unique faces detected: {len(stats['unique_faces_detected'])}")
    print(f"Total number of faces detected: {stats['total_faces_detected']}")
    print(f"\n👤 List of detected faces/IDs:")
    
    # Count occurrences of each face
    face_counts = {}
    for face_name in stats['face_names']:
        face_counts[face_name] = face_counts.get(face_name, 0) + 1
    
    if face_counts:
        for face_name, count in sorted(face_counts.items()):
            print(f"  - {face_name}: {count} occurrence(s)")
    else:
        for face_id in sorted(stats['unique_faces_detected']):
            print(f"  - {face_id}")
    
    if len(stats['unique_faces_detected']) == 0:
        print("  (No faces detected in any images)")
    
    print("="*60)

# Run the processing
process_photos_with_face_detection(INPUT_FOLDER, OUTPUT_FOLDER)



📁 Processing 4 images from People_Input...

[1/4] Processing: gonca3.jpg
  ✗ Error processing gonca3.jpg: (InvalidRequest) Invalid request has been sent.
[2/4] Processing: mada1.jpg
  ✗ Error processing mada1.jpg: (InvalidRequest) Invalid request has been sent.
[3/4] Processing: mada2.jpg
  ✗ Error processing mada2.jpg: (InvalidRequest) Invalid request has been sent.
[4/4] Processing: photo.jpg
  ✗ Error processing photo.jpg: (InvalidRequest) Invalid request has been sent.

✓ Metadata saved to: People_Output\face_metadata.json

📊 PROCESSING SUMMARY
Number of photos processed: 4
Number of unique faces detected: 0
Total number of faces detected: 0

👤 List of detected faces/IDs:
  (No faces detected in any images)


In [5]:
# ===== OPTIONAL: CELEBRITY DETECTION SETUP =====
# Requires Azure Computer Vision API (different from Face API)
# Add to .env:
# API_KEY_VISION=your_computer_vision_key
# URL_VISION=your_computer_vision_endpoint

from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes

# Load Computer Vision credentials
API_KEY_VISION = os.getenv('API_KEY_VISION')
URL_VISION = os.getenv('URL_VISION')

if API_KEY_VISION and URL_VISION:
    vision_client = ComputerVisionClient(
        URL_VISION,
        CognitiveServicesCredentials(API_KEY_VISION)
    )
    
    def detect_celebrities(image_path):
        """Detect celebrities in an image using Computer Vision API"""
        with open(image_path, 'rb') as image_stream:
            result = vision_client.analyze_image_by_domain_in_stream(
                "celebrities",
                image_stream
            )
            
            if result.result and 'celebrities' in result.result:
                return result.result['celebrities']
        return []
    
    print("✓ Celebrity detection enabled")
else:
    print("ℹ Celebrity detection not configured (Computer Vision API required)")


ℹ Celebrity detection not configured (Computer Vision API required)
